In [93]:
import tools.conf
import psycopg2
from psycopg2 import sql
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv('./data/baza.csv', delimiter=';')


In [6]:
df.head()

,data,dzien,dzien_tyg,nr_tyg,rok,miesiac,zasięg medium,brand,produkt(4),kod_reklamy,opis_reklamy,wydawca_nadawca,typ_reklamy,submedium,dl_mod,gg,mm,koszt,l_emisji,daypart,dł_ujednolicona,godzina_bloku_reklamowego
0,2023-10-01,1,Niedziela,39,2023,10,regionalne,MEDIA MASTER,GRUPA,22193792,WIESZ CO MARIAN SUPER JEST..NOWY TV..KTO GO..S...,PIGLET S.A.,reklama,YELLOW HITS Zielona Góra,29,19,43,11,1,po 16,30,19:30-19:59
1,2023-10-01,1,Niedziela,39,2023,10,ponadregionalne,MEDIA MASTER,GRUPA,22199204,"PRZECENY NA URODZINY W - SF,LAP,EKS,ODK.BP,LOD...",VOLKSDONKY,reklama,MINUS Gdańsk,15,12,42,210,1,od 9 do 16,15,12:30-12:59
2,2023-10-01,1,Niedziela,39,2023,10,ponadregionalne,MEDIA MASTER,GRUPA,22202079,TY PATRZ MARIAN DO PONIEDZIAŁKU WEEKENDOWE RAB...,VOLKSDONKY,reklama,MINUS Gdańsk,15,12,21,210,1,od 9 do 16,15,12:00-12:29
3,2023-10-01,1,Niedziela,39,2023,10,ponadregionalne,MEDIA MASTER,GRUPA,22202079,TY PATRZ MARIAN DO PONIEDZIAŁKU WEEKENDOWE RAB...,VOLKSDONKY,reklama,MINUS Gdańsk,15,11,20,210,1,od 9 do 16,15,11:00-11:29
4,2023-10-01,1,Niedziela,39,2023,10,ponadregionalne,MEDIA MASTER,GRUPA,22179830,MARIAN A GDZIE MOGĘ KUPIĆ..DOKUPIĆ..ŻEBY TO WS...,VOLKSDONKY,reklama,MINUS Gdańsk,15,10,59,210,1,od 9 do 16,15,10:30-10:59


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366681 entries, 0 to 366680
Data columns (total 22 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   data                       366681 non-null  object
 1   dzien                      366681 non-null  int64 
 2   dzien_tyg                  366681 non-null  object
 3   nr_tyg                     366681 non-null  int64 
 4   rok                        366681 non-null  int64 
 5   miesiac                    366681 non-null  int64 
 6   zasięg medium              366681 non-null  object
 7   brand                      366681 non-null  object
 8   produkt(4)                 366681 non-null  object
 9   kod_reklamy                366681 non-null  int64 
 10  opis_reklamy               366681 non-null  object
 11  wydawca_nadawca            366681 non-null  object
 12  typ_reklamy                366681 non-null  object
 13  submedium                  366681 non-null  

In [90]:
dow = df['dzien_tyg'].unique()
dow = list(dow)
dow2 = dow[1:]
dow2.append(dow[0])
dow2

['Poniedziałek',
 'Wtorek',
 'Środa',
 'Czwartek',
 'Piątek',
 'Sobota',
 'Niedziela']

In [99]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()


for day in dow2:
    cur.execute(
        sql.SQL("INSERT INTO {} (dow_name) VALUES (%s)")
        .format(sql.Identifier('pl_dow_names')), (day,)
    )

conn.commit()

In [105]:
months = ['Styczeń', 'Luty', 'Marzec', 'Kwiecień', 'Maj',
          'Czerwiec', 'Lipiec', 'Sierpień', 'Wrzesień',
          'Październik', 'Listopad', 'Grudzień']

In [107]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()


query = sql.SQL("INSERT INTO {table} ({field}) VALUES (%s)")

for month in months:
    cur.execute(
        query.format(
            table=sql.Identifier('pl_month_names'),
            field=sql.Identifier('month_name')), (month,)
    )

conn.commit()
conn.close()
